In [3]:
import pandas as pd

stock_data = pd.read_csv('/content/drive/MyDrive/Projects/stocks.csv')

stock_data

,Ticker,Date,Open,High,Low,Close,Adj Close,Volume
0,AAPL,2023-02-07,150.639999,155.229996,150.639999,154.649994,154.414230,83322600
1,AAPL,2023-02-08,153.880005,154.580002,151.169998,151.919998,151.688400,64120100
2,AAPL,2023-02-09,153.779999,154.330002,150.419998,150.869995,150.639999,56007100
3,AAPL,2023-02-10,149.460007,151.339996,149.220001,151.009995,151.009995,57450700
4,AAPL,2023-02-13,150.949997,154.259995,150.919998,153.850006,153.850006,62199000
...,...,...,...,...,...,...,...,...
243,GOOG,2023-05-01,107.720001,108.680000,107.500000,107.709999,107.709999,20926300
244,GOOG,2023-05-02,107.660004,107.730003,104.500000,105.980003,105.980003,20343100
245,GOOG,2023-05-03,106.220001,108.129997,105.620003,106.120003,106.120003,17116300
246,GOOG,2023-05-04,106.160004,106.300003,104.699997,105.209999,105.209999,19780600


In [4]:
descriptive_stats = stock_data.groupby('Ticker')['Close'].describe()
descriptive_stats

,count,mean,std,min,25%,50%,75%,max
Ticker,,,,,,,,
AAPL,62.0,158.240645,7.360485,145.309998,152.077499,158.055000,165.162506,173.570007
GOOG,62.0,100.631532,6.279464,89.349998,94.702501,102.759998,105.962503,109.459999
MSFT,62.0,275.039839,17.676231,246.270004,258.742500,275.810013,287.217506,310.649994
NFLX,62.0,327.614677,18.554419,292.760010,315.672493,325.600006,338.899994,366.829987


**Time Series Analysis**

In [5]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio


In [23]:
stock_data['Date'] = pd.to_datetime(stock_data['Date'])
pivot_data = stock_data.pivot(index='Date',columns='Ticker',values='Close')
pio.templates.default ="plotly_white"
#pio.templates.default = "plotly_white"

fig=make_subplots(rows=1,cols=1)

for columns in pivot_data.columns:
  fig.add_trace(
      go.Scatter(x=pivot_data.index,y=pivot_data[columns],name=columns),
      row=1,col=1
  )

fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Closing price for ticker',
    title_text='Volatility of Closing Prices (Standard Deviation)',
    legend_title='Ticker',
    showlegend=True)

fig.show()

**Volatility Analysis**


In [19]:
volatility = pivot_data.std().sort_values(ascending=False)
volatility

,0
Ticker,
NFLX,18.554419
MSFT,17.676231
AAPL,7.360485
GOOG,6.279464


In [21]:
fig = px.bar(
    x=volatility.index,
    y=volatility.values,
    labels={'x':'Ticker Name','y':'Standard deviation'},
    title='Volatility of Closing Prices (Standard Deviation)'
)
fig.show()

**Correlation Analysis**


In [24]:
correlation_matrix = pivot_data.corr()
correlation_matrix

Ticker,AAPL,GOOG,MSFT,NFLX
Ticker,,,,
AAPL,1.000000,0.901662,0.953037,0.154418
GOOG,0.901662,1.000000,0.884527,0.201046
MSFT,0.953037,0.884527,1.000000,0.191273
NFLX,0.154418,0.201046,0.191273,1.000000


In [29]:
fig=go.Figure(data=go.Heatmap(
        z=correlation_matrix,
        y=correlation_matrix.columns,
        x=correlation_matrix.columns,
        colorscale='blues',
        colorbar=dict(title='Correlation')
    )
)

fig.update_layout(
    title='Correlation Matrix of Closing Prices',
    xaxis_title='Ticker',
    yaxis_title='Ticker'
    )
fig.show()

**Comparative Analysis**

In [30]:
percentage_change = ((pivot_data.iloc[-1]-pivot_data.iloc[0])/pivot_data.iloc[0])*100
percentage_change

,0
Ticker,
AAPL,12.234086
GOOG,-1.689193
MSFT,16.104798
NFLX,-11.073151


In [31]:
fig=px.bar(
    x=percentage_change.index,
    y=percentage_change.values,
    labels={'x':'Tickers','y':'Percentage change in the value'},
    title = 'Percentage change in the closing values'
)
fig.show()

**Daily Risk Vs. Return Analysis**

In [32]:
pivot_data

Ticker,AAPL,GOOG,MSFT,NFLX
Date,,,,
2023-02-07,154.649994,108.040001,267.559998,362.950012
2023-02-08,151.919998,100.000000,266.730011,366.829987
2023-02-09,150.869995,95.459999,263.619995,362.500000
2023-02-10,151.009995,94.860001,263.100006,347.359985
2023-02-13,153.850006,95.000000,271.320007,358.570007
...,...,...,...,...
2023-05-01,169.589996,107.709999,305.559998,324.119995
2023-05-02,168.539993,105.980003,305.410004,317.549988
2023-05-03,167.449997,106.120003,304.399994,319.299988


In [33]:
daily_return = pivot_data.pct_change().dropna()
daily_return

Ticker,AAPL,GOOG,MSFT,NFLX
Date,,,,
2023-02-08,-0.017653,-0.074417,-0.003102,0.010690
2023-02-09,-0.006912,-0.045400,-0.011660,-0.011804
2023-02-10,0.000928,-0.006285,-0.001972,-0.041766
2023-02-13,0.018807,0.001476,0.031243,0.032272
2023-02-14,-0.004225,-0.000526,0.003133,0.003876
...,...,...,...,...
2023-05-01,-0.000530,-0.004713,-0.005533,-0.017610
2023-05-02,-0.006191,-0.016062,-0.000491,-0.020270
2023-05-03,-0.006467,0.001321,-0.003307,0.005511


In [36]:
avg_daily_return = daily_return.mean()
avg_daily_return

,0
Ticker,
AAPL,0.001991
GOOG,-0.000067
MSFT,0.002606
NFLX,-0.001676


In [37]:
risk = daily_return.std()
risk

,0
Ticker,
AAPL,0.014152
GOOG,0.020710
MSFT,0.017881
NFLX,0.022481


In [38]:
risk_return_df = pd.DataFrame({'Risk':risk,'Average_daily_return':avg_daily_return})
risk_return_df

,Risk,Average_daily_return
Ticker,,
AAPL,0.014152,0.001991
GOOG,0.020710,-0.000067
MSFT,0.017881,0.002606
NFLX,0.022481,-0.001676


In [40]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=risk_return_df['Risk'],
    y=risk_return_df['Average_daily_return'],
    mode='markers+text',
    text = risk_return_df.index,
    textposition="top center",
    marker=dict(size=10)
))

fig.update_layout(
    title = 'Risk VS Return Analysis',
    xaxis_title='Risk(Standard deviation)',
    yaxis_title='Average Daily Return '
)
fig.show()